# PRACTICA 8 

### 4.5 Levenshtein

Algorisme de teoria 

In [53]:
def levenshtein_distance(first, second):
    if len(first) > len(second): 
        first, second = second, first
    if len(second) == 0: 
        return len(first)
    first_length = len(first) + 1
    second_length = len(second) + 1
    distance_matrix = [[0] * second_length for x in range(first_length)]
    for i in range(first_length): 
        distance_matrix[i][0] = i
    for j in range(second_length): 
        distance_matrix[0][j] = j
    for i in range(1, first_length):
        for j in range(1, second_length):
            deletion = distance_matrix[i][j - 1] + 1
            insertion = distance_matrix[i - 1][j] + 1
            substitution = distance_matrix[i-1][j-1]
            if first[i-1] != second[j-1]: 
                substitution += 1
            distance_matrix[i][j] = min(insertion,deletion,substitution)
    return distance_matrix[first_length-1][second_length-1]

Algorisme modificat per la pràctica

In [2]:
def levensthein(patro, text, dlt = 2, insr = 2, subs = 1):
    
    #comprovar les longituds del patro i del text
    if len(patro) > len(text): 
        patro, text = text, patro
    if len(text) == 0: 
        return len(patro)
    
    #inicialitzem matriu
    len_patro = len(patro) + 1
    len_text = len(text) + 1
    matriu_distancia = [[0] * len_text for x in range(len_patro)]
    
    #inicialitzem una segona matriu per guardar les operacions 
    matriu_operacions = [[0] * len_text for x in range(len_patro)]
    
    #omplim amb els valors inicials
    for i in range(len_patro): 
        matriu_distancia[i][0] = i * insr #insercio te un cost = 2
    for j in range(len_text): 
        matriu_distancia[0][j] = 0 # primera fila tot amb 0 
        
        
    #recorregut resta de caselles
    for i in range(1, len_patro):   
        for j in range(1, len_text):
            deletion = matriu_distancia[i][j - 1] + dlt 
            insertion = matriu_distancia[i - 1][j] + insr 
            if patro[i - 1] != text[j - 1]:
                cost = subs
                substitution = matriu_distancia[i - 1][j - 1] + cost
            else:
                cost = 0 #les lletres coincideixen 
                substitution = matriu_distancia[i-1][j-1] + cost
            matriu_distancia[i][j] = min(insertion, deletion, substitution)
           
            
            #omplim la segona matriu amb les operacions -> punt 1.4
            if matriu_distancia[i][j] == substitution and cost == 0:
                #coincideixen 
                matriu_operacions[i][j] = 'C'
            elif matriu_distancia[i][j] == deletion:
                #elimina
                matriu_operacions[i][j] = 'D'
            elif matriu_distancia[i][j] == insertion:
                #inserta
                matriu_operacions[i][j] = 'I' 
            elif matriu_distancia[i][j] == substitution:
                #substituira
                matriu_operacions[i][j] = 'S'
      
    
    # bucle final per transformar el patro -> punt 1.5
    matriu_reversa = []
    fila = len(matriu_distancia) - 1
    
    # la posicio final sera la posicio del minim a la ultima fila
    posFinal = matriu_distancia[i].index(min(matriu_distancia[i]))
    columna = posFinal
    matriu_reversa.append(matriu_operacions[fila][columna])
    
    #recorre la matriu i guarda les operacions realitzades 
    while fila > 0 and columna > 0:
        op = matriu_operacions[fila][columna]
        
        if op == 'D':
            columna -= 1
            matriu_reversa.append('D')
        elif op == 'I':
            fila -= 1
            matriu_reversa.append('I')
        elif op == 'S':
            fila -= 1
            columna -= 1
            matriu_reversa.append('S')
        elif op == 'C':
            fila -= 1
            columna -= 1
            matriu_reversa.append('C')
    matriu_reversa.reverse()

    return min(matriu_distancia[i]), (columna, posFinal), matriu_reversa[: -1]



In [3]:
def dna(patro, fitxer = 'HUMAN-DNA.txt'):
    #linies de codi copiades del correu enviat pel professor
    with open(fitxer, 'r') as f:
        texto = f.read()
        linias = texto.split('\n')
        
    tupla_subcadena = ()
    linia = 0
    linia_text = 0
    seq_operacions = []
    distancia_linia = 99999999999999999999999999999999999999999999999
    
    for i in linias:
        distancia, tupla, seq = levensthein(patro, i)
        print(distancia, tupla, seq)
        if distancia < distancia_linia:
            tupla_subcadena = tupla
            distancia_linia = distancia
            seq_operacions = seq
            linia_text = linia
        linia += 1
    return linia_text, tupla_subcadena, distancia_linia, seq_operacions
dna("CTGGTACCAGCTGTATTAGC")            

11 (30, 49) ['C', 'S', 'S', 'C', 'S', 'C', 'C', 'S', 'S', 'I', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'C']
11 (29, 50) ['C', 'S', 'C', 'D', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'S']
11 (18, 37) ['S', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'I', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'S']
11 (10, 30) ['S', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'S']
11 (4, 22) ['C', 'C', 'C', 'S', 'C', 'C', 'S', 'I', 'C', 'I', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'S']
10 (22, 41) ['S', 'S', 'S', 'S', 'C', 'C', 'S', 'I', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S']
8 (36, 56) ['S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S']
10 (38, 55) ['C', 'I', 'C', 'S', 'C', 'I', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'I', 'C', 'C', 'C', 'C', 'C']
9 (25, 44) ['C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', '

11 (5, 24) ['S', 'C', 'S', 'C', 'S', 'C', 'S', 'I', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'S']
12 (10, 28) ['S', 'C', 'S', 'S', 'S', 'C', 'I', 'C', 'C', 'C', 'I', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'S']
11 (24, 44) ['S', 'S', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S']
11 (20, 38) ['S', 'C', 'C', 'C', 'I', 'C', 'S', 'I', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'C', 'S']
9 (2, 19) ['C', 'C', 'S', 'I', 'C', 'I', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'I', 'C', 'C', 'C', 'S', 'C', 'C']
9 (18, 37) ['S', 'C', 'S', 'S', 'C', 'I', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C']
11 (17, 37) ['S', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'C']
9 (43, 64) ['S', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'D', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'S']
10 (39, 58) ['C', 'C', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'I', 'C'

11 (14, 32) ['S', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'I', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'C', 'I', 'C']
10 (12, 33) ['S', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'D', 'C', 'C', 'C']
9 (0, 19) ['S', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'I', 'C', 'C', 'C', 'C', 'S', 'S']
11 (9, 29) ['S', 'C', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'S']
11 (11, 30) ['S', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'S', 'I', 'C']
10 (17, 38) ['C', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'D', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'C']
10 (18, 37) ['C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'S', 'C', 'S', 'I', 'C', 'C', 'C', 'C', 'S']
10 (32, 52) ['S', 'S', 'C', 'C', 'I', 'C', 'C', 'C', 'C', 'D', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S']
8 (50, 70) ['S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C

11 (18, 37) ['C', 'S', 'S', 'S', 'C', 'S', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'I', 'C', 'C', 'C', 'C', 'C', 'S']
11 (14, 33) ['S', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'I', 'C', 'C', 'C', 'S', 'S']
10 (52, 70) ['C', 'C', 'S', 'S', 'C', 'S', 'I', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'I', 'C', 'C', 'C', 'S']
9 (32, 51) ['C', 'S', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'I', 'C', 'C']
10 (33, 53) ['S', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S']
10 (33, 52) ['S', 'C', 'S', 'S', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'I', 'C']
11 (31, 51) ['S', 'S', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S']
9 (32, 51) ['C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'I', 'C', 'S', 'C', 'C', 'S']
10 (20, 39) ['C', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'I', 'C', 'C', 

10 (12, 32) ['C', 'C', 'S', 'C', 'S', 'S', 'C', 'D', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'I', 'C', 'C', 'S', 'C', 'S']
10 (6, 25) ['S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'I', 'C', 'S', 'C', 'C', 'S']
10 (50, 70) ['C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C']
8 (6, 24) ['C', 'C', 'C', 'S', 'C', 'C', 'S', 'I', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'I', 'C']
11 (0, 19) ['C', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S']
11 (35, 54) ['S', 'C', 'S', 'S', 'C', 'S', 'I', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S']
10 (41, 61) ['S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'C']
11 (9, 31) ['C', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'D', 'D', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S']
10 (9, 29) ['S', 'C', 'C', 'S', 'S', 'C', 'S', 'I', 'C', 'C', 'S', 'C', 'C', 'S', 'D', 'C',

10 (35, 55) ['S', 'S', 'S', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C']
10 (16, 36) ['S', 'C', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'C']
12 (4, 22) ['S', 'S', 'C', 'C', 'C', 'S', 'S', 'I', 'C', 'C', 'I', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C']
10 (0, 20) ['C', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'S', 'C']
9 (15, 35) ['C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'S']
9 (20, 39) ['C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'I', 'C', 'S', 'S', 'C', 'S']
11 (14, 34) ['C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'S']
12 (6, 24) ['S', 'C', 'S', 'S', 'C', 'C', 'S', 'I', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'I', 'C', 'C', 'S', 'S']
9 (34, 54) ['C', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S',

9 (18, 38) ['S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'D', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'I', 'C']
10 (49, 68) ['S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'I', 'C', 'S', 'C', 'C', 'S']
10 (31, 52) ['C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'D', 'C', 'C', 'C', 'C', 'S', 'C']
10 (4, 24) ['C', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'C']
11 (29, 48) ['S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'I', 'C', 'C', 'S', 'S']
10 (8, 27) ['S', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'I', 'C', 'C', 'S', 'C', 'C', 'S', 'S']
8 (37, 58) ['S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'D', 'C', 'S', 'C', 'C', 'C', 'C']
11 (1, 18) ['S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'I', 'C', 'C', 'C', 'I', 'C', 'S', 'I', 'C', 'C']
10 (40, 59) ['S', 'C', 'C', 'C', 'S', 'C', 'S', 'I', 'C', 'S', 'C', 'S', 'C', 'S', 'C'

11 (37, 56) ['S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'S', 'I', 'C', 'S', 'C', 'C', 'S', 'S', 'C']
11 (32, 52) ['C', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S']
9 (21, 39) ['S', 'C', 'C', 'C', 'S', 'C', 'I', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'I', 'C']
11 (13, 34) ['S', 'S', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'D', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C']
10 (7, 28) ['S', 'C', 'S', 'C', 'C', 'C', 'D', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'S']
10 (46, 64) ['C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'I', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'I', 'C']
12 (14, 33) ['S', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'C', 'I', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C']
10 (22, 40) ['C', 'S', 'C', 'S', 'C', 'I', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'I', 'C', 'C', 'C', 'S']
11 (6, 25) ['C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'S', 'I', 'C', '

10 (1, 21) ['S', 'S', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S']
11 (6, 25) ['C', 'C', 'S', 'I', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'C']
12 (16, 34) ['S', 'C', 'S', 'I', 'C', 'C', 'C', 'S', 'S', 'C', 'I', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S']
11 (22, 41) ['S', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S', 'S', 'C', 'I', 'C', 'C', 'C', 'S', 'C', 'S', 'C']
10 (5, 25) ['S', 'C', 'C', 'D', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'I', 'C', 'C', 'S', 'C']
10 (9, 27) ['C', 'S', 'S', 'I', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'I', 'C', 'C', 'C', 'C', 'C', 'S', 'C']
10 (17, 35) ['C', 'C', 'S', 'S', 'C', 'C', 'S', 'I', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'I', 'C', 'C']
8 (37, 58) ['C', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'C', 'D', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
10 (9, 30) ['C', 'S', 'S', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'D', 'C', 'C', 'C', 'S',

(728,
 (11, 31),
 6,
 ['C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'C',
  'S',
  'C',
  'S',
  'C',
  'S',
  'S',
  'S',
  'C',
  'C',
  'S',
  'C',
  'C',
  'C'])

In [106]:
assert dna("CTGGTACCAGCTGTATTAGC") == (728, (11, 31), 6, ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("TCGTCATAAACCGCTGTGCC") == (212, (12, 32), 7, ['S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S'])

In [107]:
assert dna("TATACAAACGGAGTAGCTGT") == (285, (5, 25), 6, ['C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("AGGCGTAAGTCTTACGTATA") == (5, (41, 61), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C'])

In [108]:
assert dna("AACGGCATAGCCTGCAAGAG") == (433, (41, 61), 5, ['C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S'])
assert dna("GTGCGTCCACCCTTAATACA") == (196, (41, 61), 6, ['C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])

In [109]:
assert dna("GTCAGCACCGGGATCTGTAT") == (240, (26, 46), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])
assert dna("GAGCCCCGACGTTTTAACGA") == (68, (6, 26), 7, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C'])
assert dna("CCTCGTAGACAGTACCGAAT") == (448, (30, 50), 6, ['C', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C'])
assert dna("CGACCAAAGAGCCTGTATCT") == (320, (35, 55), 7, ['S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("CGTGGTGTCCATACCCTAGC") == (835, (24, 43), 6, ['C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'I', 'C', 'S', 'C'])
assert dna("GTGATAGACCTTTTAAGCTG") == (409, (18, 37), 6, ['S', 'C', 'C', 'C', 'C', 'C', 'I', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C'])

In [110]:
assert dna("GACACACACTTGGATCTTCG") == (565, (16, 36), 6, ['C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("CTGGACGTTCATACATAGAC") == (28, (21, 41), 7, ['C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S'])
assert dna("ACGTTTTACCCCAAAGCCCG") == (753, (4, 24), 7, ['C', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C'])
assert dna("CGGGTAGAAATCTCCGCTTG") == (361, (30, 50), 6, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'C', 'C'])
print("All tests passed!")

All tests passed!
